In [2]:
import pandas as pd
import sqlite3
import altair as alt

# Crop Production

In [4]:
db_name = 'field_crops.db'
table = 'midwest_key_field_crops_cleaned'
query = f"""
Select 
    commodity_desc, 
    state_alpha, 
    short_desc, 
    prodn_practice_desc, 
    year, 
    sum(value) as total_prod, 
    count(*) 
from {table} 
where commodity_desc = 'CORN'
group by 
    commodity_desc, state_alpha, short_desc, year
"""

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)
result

,commodity_desc,state_alpha,short_desc,prodn_practice_desc,year,total_prod,count(*)
0,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1960,772541000.0,99
1,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1961,753188000.0,99
2,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1962,745129000.0,99
3,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1963,851600000.0,99
4,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1964,754695000.0,99
...,...,...,...,...,...,...,...
1301,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2019,18200000.0,34
1302,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2020,20160000.0,63
1303,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2021,18920000.0,64
1304,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2022,19575000.0,64


In [9]:
def silage_conversion(row, short_desc_cat, factor):
    if row['short_desc'] == short_desc_cat:
        return row['total_prod'] * factor  # Multiply by the factor for Corn as a Grain
    else:
        return row['total_prod']  # Keep the value unchanged

result['total_prod'] = result.apply(lambda row: silage_conversion(row, short_desc_cat="CORN, SILAGE - PRODUCTION, MEASURED IN TONS", factor=37), axis=1)
result

,commodity_desc,state_alpha,short_desc,prodn_practice_desc,year,total_prod,count(*)
0,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1960,772541000.0,99
1,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1961,753188000.0,99
2,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1962,745129000.0,99
3,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1963,851600000.0,99
4,CORN,IA,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",ALL PRODUCTION PRACTICES,1964,754695000.0,99
...,...,...,...,...,...,...,...
1301,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2019,673400000.0,34
1302,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2020,745920000.0,63
1303,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2021,700040000.0,64
1304,CORN,WI,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",ALL PRODUCTION PRACTICES,2022,724275000.0,64


In [ ]:
db_name = 'field_crops.db'
table = 'midwest_key_field_crops_cleaned'
query = f"Select commodity_desc, state_alpha, short_desc, prodn_practice_desc, year, sum(value) as total_prod, count(*) from {table} group by commodity_desc, state_alpha, short_desc, year"

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)
result['total_prod'] = result.apply(lambda row: silage_conversion(row, short_desc_cat="CORN, SILAGE - PRODUCTION, MEASURED IN TONS", factor=37), axis=1)

#pd.set_option('display.max_rows', None)  # Display all rows

midwestern_states = ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']
good_data_commodities = [ 'CORN', 'SOYBEANS', 'WHEAT']

for commodity in good_data_commodities:
    for state in midwestern_states:
        df = result[result['commodity_desc'] == commodity]
        df = df[df['state_alpha'] == state]

        # Create the Altair line chart
        chart = alt.Chart(df).mark_line().encode(
        x='year:O',  # Treat 'year' as an ordinal value (categorical)
        y='total_prod:Q',
        color='short_desc:N'
        ).properties(
            title=f'Total Production in {state} by {commodity} Over Time',
            width=600,
            height=400
        )

        # Display the chart
        chart.show()

# Area Planted

In [2]:
db_name = 'field_crops.db'
table = 'midwest_area_planted_cleaned'
query = f"Select commodity_desc, state_alpha, short_desc, prodn_practice_desc, year, sum(value) as total_prod, count(*) from {table} group by commodity_desc, state_alpha, short_desc, year"

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)
#pd.set_option('display.max_rows', None)  # Display all rows

midwestern_states = ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']
good_data_commodities = [ 'CORN', 'SOYBEANS', 'WHEAT']

for commodity in good_data_commodities:
    for state in midwestern_states:
        df = result[result['commodity_desc'] == commodity]
        df = df[df['state_alpha'] == state]

        # Create the Altair line chart
        chart = alt.Chart(df).mark_line().encode(
        x='year:O',  # Treat 'year' as an ordinal value (categorical)
        y='total_prod:Q',
        color='short_desc:N'
        ).properties(
            title=f'Total Production in {state} by {commodity} Over Time',
            width=600,
            height=400
        )

        # Display the chart
        chart.show()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [ ]:
db_name = 'field_crops.db'
table = 'midwest_area_planted_cleaned'
query = f"Select commodity_desc, state_alpha, short_desc, prodn_practice_desc, year, sum(value) as total_prod, count(*) from {table} group by commodity_desc, state_alpha, short_desc, year"

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)
pd.set_option('display.max_rows', None)  # Display all rows
result

unique_commodities = result['commodity_desc'].unique()

midwestern_states = ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']
good_data_commodities = [ 'CORN', 'SOYBEANS', 'WHEAT']

for commodity in good_data_commodities:
    for state in midwestern_states:
        df = result[result['commodity_desc'] == commodity]
        df = df[df['state_alpha'] == state]
        #df = df[df['prodn_practice_desc'] == 'ALL PRODUCTION PRACTICES']

        #df = df[df['year'] <= 2018]
        # Create the Altair line chart
        chart = alt.Chart(df).mark_line().encode(
        x='year:O',  # Treat 'year' as an ordinal value (categorical)
        y='total_prod:Q',
        color='short_desc:N'
        ).properties(
            title=f'Total Production in {state} by {commodity} Over Time',
            width=600,
            height=400
        )

        df_2018 = df[df['year'] == 2023]
        print(df_2018.groupby(['short_desc'])['total_prod'].sum().reset_index(name='Sum_Value'))



        # Display the chart
        chart.show()

In [ ]:
db_name = 'field_crops.db'
table = 'midwest_area_planted_cleaned'
query = f"Select commodity_desc, state_alpha, short_desc, prodn_practice_desc, year, sum(value) as total_prod, count(*) from {table} group by commodity_desc, state_alpha, short_desc, year"

conn = sqlite3.connect(db_name) 
result = pd.read_sql(query, conn)
pd.set_option('display.max_rows', None)  # Display all rows
result

unique_commodities = result['commodity_desc'].unique()

midwestern_states = ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']
good_data_commodities = [ 'CORN', 'SOYBEANS', 'WHEAT']

for commodity in good_data_commodities:
    for state in midwestern_states:
        df = result[result['commodity_desc'] == commodity]
        df = df[df['state_alpha'] == state]
        #df = df[df['prodn_practice_desc'] == 'ALL PRODUCTION PRACTICES']

        #df = df[df['year'] <= 2018]
        # Create the Altair line chart
        chart = alt.Chart(df).mark_line().encode(
        x='year:O',  # Treat 'year' as an ordinal value (categorical)
        y='total_prod:Q',
        color='short_desc:N'
        ).properties(
            title=f'Total Production in {state} by {commodity} Over Time',
            width=600,
            height=400
        )

        df_2018 = df[df['year'] == 2023]
        print(df_2018.groupby(['short_desc'])['total_prod'].sum().reset_index(name='Sum_Value'))



        # Display the chart
        chart.show()